Auto Improvement Using MetaModelClassification - Example Usage
===

This notebook demonstrates two example usages of MetaModelClassification:

Example 1: Bank Marketing Dataset. <br>
Example 2: German Credit Risk Dataset. <br>

Read about the datasets [here](../../Docs/Data_Dictionaries/ReadMe.md) and their baselines [here](../../Docs/Models/Baseline).

In [ ]:
from auto_improve_pipeline import MetamodelClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss

# Example 1: Improving Log Loss on the Bank Marketing Dataset

# Example 2: Improving Log Loss on the German Credit Risk Dataset